# LFM2-1.2B v6 SFT Training (200k + 80k Curriculum) - FIXED

**v6 Training Pipeline**:
1. **Phase 1**: 200k 전체 SFT (Base → v6-SFT-200k)
2. **Phase 2**: 80k 고품질 SFT (v6-SFT-200k → v6.1-SFT-80k)

---

### 🔧 디스크 최적화 (FIXED)
1. **캐시를 /tmp로 이동**: `/kaggle/working`은 20GB 제한! → `/tmp`는 별도 용량
2. **체크포인트 저장 빈도 감소**: 500 → **2000** steps
3. **체크포인트 수 제한**: 3 → 2개

### 🌟 Kaggle 설정 (필수)
1. **Accelerator**: **GPU T4 x 2**
2. **Internet**: **On**
3. **Secrets**: `HF_TOKEN` 설정 (Hugging Face Write 권한 토큰)
4. **Dataset**: HuggingFace `gyung/lfm2-koen-samples` (자동 다운로드)

### ⏱️ 예상 시간
- Phase 1 (200k): ~8-10시간
- Phase 2 (80k): ~3-4시간
- **총: ~12-14시간**

In [ ]:
# 1. 캐시 디렉토리 설정 (/tmp 사용 - 디스크 절약 핵심!)
import os
os.environ["HF_HOME"] = "/tmp/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "/tmp/hf_cache/datasets"
os.environ["TRANSFORMERS_CACHE"] = "/tmp/hf_cache/transformers"

# 2. 필수 패키지 설치
!pip install -q transformers==4.54.0 trl>=0.18.2 peft>=0.15.2 datasets accelerate bitsandbytes huggingface_hub protobuf==3.20.3

from kaggle_secrets import UserSecretsClient

# 3. Hugging Face 토큰 가져오기
try:
    user_secrets = UserSecretsClient()
    os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
    print("✅ HF_TOKEN 환경 변수 설정 완료!")
except Exception as e:
    print(f"⚠️ 토큰 설정 실패: {e}")
    print("Add-ons -> Secrets 에서 HF_TOKEN을 설정했는지 확인하세요.")

In [ ]:
%%writefile train_v6_fixed.py
import os
import shutil
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, SFTConfig, setup_chat_format
from huggingface_hub import login

def find_dataset(target_files):
    """Kaggle input에서 데이터셋 파일 찾기"""
    for root, dirs, files in os.walk("/kaggle/input"):
        for target in target_files:
            if target in files:
                return os.path.join(root, target)
    return None

def create_mt_dataset(repo_id, config_name):
    """HuggingFace에서 데이터셋 로드"""
    print(f"📚 Loading: {repo_id} ({config_name})")
    base_dataset = load_dataset(repo_id, config_name, split="train")
    
    def format_en2ko(row):
        return {"messages": [{"role":"system","content":"Translate to Korean."}, 
                            {"role":"user","content":row['input']}, 
                            {"role":"assistant","content":row['output']}]}
    def format_ko2en(row):
        return {"messages": [{"role":"system","content":"Translate to English."}, 
                            {"role":"user","content":row['output']}, 
                            {"role":"assistant","content":row['input']}]}
    
    ds_en2ko = base_dataset.map(format_en2ko, remove_columns=base_dataset.column_names, num_proc=4)
    ds_ko2en = base_dataset.map(format_ko2en, remove_columns=base_dataset.column_names, num_proc=4)
    full_dataset = concatenate_datasets([ds_en2ko, ds_ko2en])
    return full_dataset.shuffle(seed=42)

def cleanup_checkpoints(output_dir):
    """로컬 체크포인트 삭제하여 디스크 공간 확보"""
    if os.path.exists(output_dir):
        print(f"🧹 Cleaning up: {output_dir}")
        shutil.rmtree(output_dir, ignore_errors=True)

def train_phase(model_id, dataset_repo, dataset_config, output_name, hub_model_id, hf_token, phase_name):
    """한 Phase 학습 실행"""
    print(f"\n{'='*60}")
    print(f"🚀 {phase_name}")
    print(f"{'='*60}")
    print(f"📦 Model: {model_id}")
    print(f"📊 Dataset: {dataset_repo} ({dataset_config})")
    print(f"💾 Output: {hub_model_id}")
    
    # 모델 로드
    print("\n🔹 Loading Model...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="float16")
    
    if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)
    
    # 데이터셋 준비
    print("🔹 Preparing Dataset...")
    train_dataset = create_mt_dataset(dataset_repo, dataset_config)
    print(f"   Total samples: {len(train_dataset):,}")
    
    # 학습 설정 (디스크 최적화)
    output_dir = f"/kaggle/working/{output_name}"
    sft_config = SFTConfig(
        output_dir=output_dir,
        hub_model_id=hub_model_id,
        hub_token=hf_token,
        push_to_hub=True,
        num_train_epochs=1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        gradient_checkpointing=True,
        optim="paged_adamw_8bit",
        fp16=False,
        bf16=False,
        learning_rate=1e-5,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        logging_steps=250,          # 로그는 250 step마다
        save_strategy="steps",
        save_steps=1000,             # ✅ 1000 step마다 저장 (디스크 절약)
        save_total_limit=2,          # ✅ 최대 2개만 유지
        eval_strategy="no",          # 평가 생략 (속도)
        dataset_text_field="messages",
        packing=False,
        report_to="none",
        ddp_find_unused_parameters=False,
    )
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        args=sft_config,
        processing_class=tokenizer,
    )
    
    print("🚀 Starting Training...")
    trainer.train()
    
    if trainer.is_world_process_zero():
        print("💾 Final Saving & Uploading to Hub...")
        trainer.save_model(output_dir)
        tokenizer.save_pretrained(output_dir)
        trainer.push_to_hub()
        try:
            tokenizer.push_to_hub(hub_model_id)
        except:
            pass
        print(f"✅ {phase_name} Complete!")
    
    # 메모리 정리
    del model, trainer
    torch.cuda.empty_cache()
    
    # 디스크 정리 (Phase 완료 후)
    cleanup_checkpoints(output_dir)
    
    return hub_model_id

def main():
    hf_token = os.environ.get("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
    else:
        print("⚠️ HF_TOKEN not found. Uploads will fail.")
        return
    
    os.environ["WANDB_DISABLED"] = "true"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    
    # ============================================
    # ⚙️ 설정 (여기만 수정하면 됨)
    # ============================================
    BASE_MODEL = "LiquidAI/LFM2-1.2B"
    HUB_USERNAME = "gyung"
    
    # HuggingFace 데이터셋 설정
    DATASET_REPO = "gyung/lfm2-koen-samples"
    PHASE1_CONFIG = "manual_1000_sft"
    PHASE1_OUTPUT = "lfm2-1.2b-koen-mt-v6-sft-200k"
    
    PHASE2_CONFIG = "manual_1000_sft"  # 같은 데이터셋 사용
    PHASE2_OUTPUT = "lfm2-1.2b-koen-mt-v6.1-sft-80k"
    # ============================================
    
    # HuggingFace에서 자동 다운로드
    print(f"📚 Dataset: {DATASET_REPO}")
    
    print("\n" + "="*60)
    print("🎯 v6 SFT Training Pipeline")
    print("="*60)
    print(f"Phase 1: {DATASET_REPO}/{PHASE1_CONFIG} → {PHASE1_OUTPUT}")
    print(f"Phase 2: {DATASET_REPO}/{PHASE2_CONFIG} → {PHASE2_OUTPUT}")
    
    # Phase 1: 200k 학습
    phase1_model = train_phase(
        model_id=BASE_MODEL,
        dataset_repo=DATASET_REPO,
        dataset_config=PHASE1_CONFIG,
        output_name=PHASE1_OUTPUT,
        hub_model_id=f"{HUB_USERNAME}/{PHASE1_OUTPUT}",
        hf_token=hf_token,
        phase_name="Phase 1: SFT 200k"
    )
    
    # Phase 2: 80k 고품질 학습 (Phase 1 결과에서 이어서)
    train_phase(
        model_id=phase1_model,  # Phase 1 결과 모델에서 시작
        dataset_repo=DATASET_REPO,
        dataset_config=PHASE2_CONFIG,
        output_name=PHASE2_OUTPUT,
        hub_model_id=f"{HUB_USERNAME}/{PHASE2_OUTPUT}",
        hf_token=hf_token,
        phase_name="Phase 2: SFT 80k (Curriculum)"
    )
    
    print("\n" + "="*60)
    print("🎉 All Phases Complete!")
    print("="*60)
    print(f"✅ v6-SFT-200k: https://huggingface.co/{HUB_USERNAME}/{PHASE1_OUTPUT}")
    print(f"✅ v6.1-SFT-80k: https://huggingface.co/{HUB_USERNAME}/{PHASE2_OUTPUT}")

if __name__ == "__main__":
    main()

In [ ]:
# 🚀 학습 실행 (T4 x 2 DDP)
!accelerate launch --multi_gpu --num_processes 2 --mixed_precision no train_v6_fixed.py

In [ ]:
# ⛑️ Rescue Cell (학습 중단 시 수동 업로드)
import os
from huggingface_hub import HfApi, login
from kaggle_secrets import UserSecretsClient

# 로그인
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

# 업로드할 모델 선택 (경로 확인 후 수정)
MODELS_TO_UPLOAD = {
    "/kaggle/working/lfm2-1.2b-koen-mt-v6-sft-200k": "gyung/lfm2-1.2b-koen-mt-v6-200k",
    "/kaggle/working/lfm2-1.2b-koen-mt-v6.1-sft-80k": "gyung/lfm2-1.2b-koen-mt-v6.1-sft-80k",
}

api = HfApi()
for local_path, repo_id in MODELS_TO_UPLOAD.items():
    if os.path.exists(local_path):
        print(f"🚀 Uploading {local_path} → {repo_id}")
        try:
            api.upload_folder(folder_path=local_path, repo_id=repo_id, repo_type="model")
            print(f"✅ Done: {repo_id}")
        except Exception as e:
            print(f"❌ Failed: {e}")
    else:
        print(f"⚠️ Not found: {local_path}")

In [ ]:
# 🧪 학습된 모델 테스트
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 테스트할 모델 선택
MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6.1-sft-80k"  # 최종 모델

print(f"🔹 Loading: {MODEL_ID}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype="float16", device_map="auto")

def translate(text, to="ko"):
    system = "Translate to Korean." if to == "ko" else "Translate to English."
    messages = [{"role": "system", "content": system}, {"role": "user", "content": text}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.3,
            min_p=0.15,
            repetition_penalty=1.05,
        )
    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

# 테스트
tests = [
    "Please fix this error.",
    "The model is working correctly now.",
    "Time flies like an arrow.",  # 관용구 테스트
]

for text in tests:
    result = translate(text, to="ko")
    print(f"📝 EN: {text}")
    print(f"🇰🇷 KO: {result}")
    print("-" * 40)